In [ ]:
import google.generativeai as genai
import gradio as gr
import requests
from bs4 import BeautifulSoup
import json
from google.colab import userdata

In [ ]:
try:
    genai.configure(api_key=userdata.get('GEMINI_API_KEY'))
    model = genai.GenerativeModel('gemini-1.5-flash')
    print("✅ Gemini API configured successfully.")
except (userdata.SecretNotFoundError, Exception) as e:
    print("🛑 Error: Secret 'GEMINI_API_KEY' not found or invalid. Please add it to the Colab Secrets Manager.")
    model = None

def scrape_pune_news():
    url = "https://timesofindia.indiatimes.com/rssfeeds/-2128821991.cms"
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'xml')
        articles = []
        for item in soup.find_all('item')[:5]:
            title = item.title.string
            link = item.link.string
            articles.append({'title': title, 'link': link})
        return articles
    except requests.exceptions.RequestException as e:
        print(f"Error fetching RSS feed: {e}")
        return []

def fetch_article_text(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        paragraphs = soup.find_all('p')
        article_text = ' '.join([p.get_text() for p in paragraphs])
        return article_text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching article content from {url}: {e}")
        return None

def analyze_article_with_gemini(article_text):
    if not article_text or not model:
        return None
    categories = ["Local News", "Politics", "Business", "Sports", "Technology", "Entertainment", "Lifestyle"]
    prompt = f"""
    Analyze the following news article. Your tasks are:
    1.  Provide a concise, one-paragraph summary of the article.
    2.  Classify the article into ONE of the following categories: {', '.join(categories)}.

    You MUST return the output ONLY as a single, valid JSON object with two keys: "summary" and "category".
    Do not include any other text, markdown formatting (like ```json), or explanations.

    Article Text:
    ---
    {article_text[:4000]}
    ---
    """
    try:
        response = model.generate_content(prompt)
        cleaned_response = response.text.strip().replace('```json', '').replace('```', '')
        result = json.loads(cleaned_response)
        return result
    except (json.JSONDecodeError, Exception) as e:
        print(f"Error processing with Gemini or parsing JSON: {e}")
        return {"summary": "Analysis failed for this article.", "category": "Uncategorized"}

def create_news_digest(progress=gr.Progress()):
    if not model:
        return "## 🛑 Error\n\nGemini API is not configured. Please set the `GEMINI_API_KEY` in Colab secrets and restart the runtime."
    progress(0, desc="Fetching news headlines...")
    articles = scrape_pune_news()
    if not articles:
        return "## ⚠️ No Articles Found\n\nCould not fetch news articles. The RSS feed might be down or the URL is incorrect."
    category_emojis = {
        "Local News": "🏙️", "Politics": "🏛️", "Business": "💼",
        "Sports": "⚽", "Technology": "💻", "Entertainment": "🎬",
        "Lifestyle": "🥗", "Uncategorized": "❓", "Retrieval Error": "❌"
    }
    current_date = __import__('datetime').date.today().strftime('%B %d, %Y')
    markdown_output = f"<div style='text-align:center'><h1>📰 Pune News Digest</h1><p><em>Your daily brief for {current_date}</em></p></div>\n\n---\n"
    total_articles = len(articles)
    for i, article in enumerate(articles):
        progress((i) / total_articles, desc=f"Analyzing: {article['title'][:30]}...")
        full_text = fetch_article_text(article['link'])
        analysis = {"summary": "Could not retrieve article content.", "category": "Retrieval Error"}
        if full_text:
            analysis = analyze_article_with_gemini(full_text)
        category = analysis.get('category', 'Uncategorized')
        emoji = category_emojis.get(category, "❓")
        markdown_output += f"### {emoji} {article['title']}\n"
        markdown_output += f"**Category:** `{category}`\n"
        markdown_output += f"**Summary:** {analysis.get('summary', 'N/A')}\n\n"
        markdown_output += f"<p style='text-align:right;'><a href='{article['link']}' target='_blank'>Read Full Article ➔</a></p>\n\n"
        markdown_output += "---\n"
    progress(1, desc="Done!")
    return markdown_output

with gr.Blocks(theme=gr.themes.Default(primary_hue="blue", secondary_hue="blue"), title="Local News Digest") as demo:
    gr.Markdown(
        """
        <div style='text-align:center;'>
            <h1 style='color:#2563eb;'>Local News Digest AI 🤖</h1>
            <p>Your one-click solution to get the latest summarized and categorized news from Pune, powered by Google Gemini.</p>
        </div>
        """
    )
    with gr.Row():
        start_button = gr.Button("🚀 Generate Today's Digest", variant="primary", scale=1)
    with gr.Accordion("📰 Your News Digest", open=False) as output_accordion:
        output_markdown = gr.Markdown(label="Digest")

    def run_and_open_accordion(progress=gr.Progress()):
        results = create_news_digest(progress)
        return {
            output_accordion: gr.Accordion(open=True),
            output_markdown: results
        }

    start_button.click(
        fn=run_and_open_accordion,
        inputs=None,
        outputs=[output_accordion, output_markdown]
    )

demo.launch(debug=True)


✅ Gemini API configured successfully.
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://39ea6af65c682889f5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
